In [408]:
import pyhealth
import pandas as pd
from typing import *

from pyhealth.models import Transformer, RNN


In [409]:
radiolung_df = pd.read_excel("20230713_BDMetaData_CanRuti.xlsx")

In [ ]:
def collate_fn_dict(batch):
    return {key: [d[key] for d in batch] for key in batch[0]}



class RadioLungDataset():
    
    def __init__(self, path_df: str):
        
        self.df = pd.read_excel(path_df)
        self.input_info = {} # com estan cada feature_key
        self.map_feature_token_space()
        
    def __len__(self):
        return self.df.shape[0]
    
    def map_feature_token_space(self) -> None:
        feature_keys = self.df.columns
        for feature_key in feature_keys:
            token_space = self.df[feature_key].unique().tolist() #token_space = self.df[feature_key].unique().flatten().tolist()
            if pd.api.types.is_string_dtype(self.df[feature_key] ) == True:
                self.input_info[feature_key] = {"type": str, "dim": 2} 
            else:
                self.input_info[feature_key] = {"type": int, "dim": 2, 'len':1}# ha petat i tb i he ficat len 
                print(self.df[feature_key])
                self.df[feature_key]= (self.df[feature_key].apply(lambda x: [x])) # he ficat tots es num dins una llista pq vol aquest format
                print(self.df[feature_key])
           #todo: modificar pq puguis ficar aquesta linia enlloc de int/float -->  (type(self.df[feature_key].iloc[0]).__name__)


    def case1_case2_binary(self):
        feature_bin = ["air_pollution", "ph_cancer", "COPD", "ASTHMA", "bronchiectasis", "TB", "pneumonia"]
        
        dic_bi = {}
        for i, feature in enumerate(feature_bin):
            dic_bi[feature] = {"0": i * 2, "1": i * 2 + 1}
        
        for feature in feature_bin:
            self.df[feature] = self.df[feature].map(dic_bi[feature])
        print(self.df['TB'])
        return dic_bi
      
         
    def get_all_tokens(self, key:str):
        return self.df[key].unique().flatten().tolist()
    
    
    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        sample_return = {fkey: [value] if fkey != "type" else value for fkey, value in sample.items()}
        sample_return.update({"patient_id": sample_return["id"]})
        sample_return.pop("id")
        return sample_return

In [411]:
dataset = RadioLungDataset(path_df="20230713_BDMetaData_CanRuti.xlsx")
(dataset[0]["patient_id"][0])


 id
0       1
1       8
2       9
3      10
4      11
     ... 
94    157
95    157
96    160
97    161
98    162
Name: id, Length: 99, dtype: int64
0       [1]
1       [8]
2       [9]
3      [10]
4      [11]
      ...  
94    [157]
95    [157]
96    [160]
97    [161]
98    [162]
Name: id, Length: 99, dtype: object

 hospital

 birthdate
0    1945-07-22
1    1942-01-08
2    1949-05-14
3    1943-06-07
4    1943-06-27
        ...    
94   1956-05-29
95   1956-05-29
96   1964-07-20
97   2023-06-13
98   1943-03-10
Name: birthdate, Length: 99, dtype: datetime64[ns]
0     [1945-07-22 00:00:00]
1     [1942-01-08 00:00:00]
2     [1949-05-14 00:00:00]
3     [1943-06-07 00:00:00]
4     [1943-06-27 00:00:00]
              ...          
94    [1956-05-29 00:00:00]
95    [1956-05-29 00:00:00]
96    [1964-07-20 00:00:00]
97    [2023-06-13 00:00:00]
98    [1943-03-10 00:00:00]
Name: birthdate, Length: 99, dtype: object

 sex

 education

 BMI
0     33.0
1     33.0
2     27.0
3      0.0
4      0.0
  

[1]

In [412]:
from pyhealth.datasets import SampleEHRDataset

samples = [
    {
        "patient_id": "patient-0",
        "visit_id": "visit-0",
        "single_vector": [1, 2, 3],
        "list_codes": ["505800458"],  # NDC
        "list_vectors": [[1.0, 2.55, 3.4], [4.1, 5.5, 6.0]],
        "list_list_codes": [["A05B", "A05C", "A06A"], ["A11D", "A11E"]],  # ATC-4
        "list_list_vectors": [
            [[1.8, 2.25, 3.41], [4.50, 5.9, 6.0]],
            [[7.7, 8.5, 9.4]],
        ],
        "label": 1,
    },
    {
        "patient_id": "patient-0",
        "visit_id": "visit-1",
        "single_vector": [1, 5, 8],
        "list_codes": [
            "55154191800",
            "551541928",
            "55154192800",
            "705182798",
            "70518279800",
        ],
        "list_vectors": [[1.4, 3.2, 3.5], [4.1, 5.9, 1.7], [4.5, 5.9, 1.7]],
        "list_list_codes": [["A04A", "B035", "C129"]],
        "list_list_vectors": [
            [[1.0, 2.8, 3.3], [4.9, 5.0, 6.6], [7.7, 8.4, 1.3], [7.7, 8.4, 1.3]],
        ],
        "label": 0,
    },
]

# dataset
dataset = SampleEHRDataset(samples=samples, dataset_name="test")

# data loader
from pyhealth.datasets import get_dataloader

train_loader = get_dataloader(dataset, batch_size=2, shuffle=True)

# model
model = Transformer(
    dataset=dataset,
    feature_keys=[
        "list_codes",
        "list_vectors",
        "list_list_codes",
        "list_list_vectors",
    ],
    label_key="label",
    mode="multiclass",
)

# data batch
data_batch = next(iter(train_loader))

print(data_batch)

# try the model
ret = model(**data_batch)
print(ret)

# try loss backward
ret["loss"].backward()

{'patient_id': ['patient-0', 'patient-0'], 'visit_id': ['visit-1', 'visit-0'], 'single_vector': [[1, 5, 8], [1, 2, 3]], 'list_codes': [['55154191800', '551541928', '55154192800', '705182798', '70518279800'], ['505800458']], 'list_vectors': [[[1.4, 3.2, 3.5], [4.1, 5.9, 1.7], [4.5, 5.9, 1.7]], [[1.0, 2.55, 3.4], [4.1, 5.5, 6.0]]], 'list_list_codes': [[['A04A', 'B035', 'C129']], [['A05B', 'A05C', 'A06A'], ['A11D', 'A11E']]], 'list_list_vectors': [[[[1.0, 2.8, 3.3], [4.9, 5.0, 6.6], [7.7, 8.4, 1.3], [7.7, 8.4, 1.3]]], [[[1.8, 2.25, 3.41], [4.5, 5.9, 6.0]], [[7.7, 8.5, 9.4]]]], 'label': [0, 1]}
{'loss': tensor(5.2310, grad_fn=<NllLossBackward0>), 'y_prob': tensor([[0.0017, 0.9983],
        [0.9833, 0.0167]], grad_fn=<SoftmaxBackward0>), 'y_true': tensor([0, 1]), 'logit': tensor([[-2.8407,  3.5289],
        [ 2.5779, -1.4958]], grad_fn=<AddmmBackward0>)}


In [413]:
dataset = RadioLungDataset(path_df="20230713_BDMetaData_CanRuti.xlsx")



 id
0       1
1       8
2       9
3      10
4      11
     ... 
94    157
95    157
96    160
97    161
98    162
Name: id, Length: 99, dtype: int64
0       [1]
1       [8]
2       [9]
3      [10]
4      [11]
      ...  
94    [157]
95    [157]
96    [160]
97    [161]
98    [162]
Name: id, Length: 99, dtype: object

 hospital

 birthdate
0    1945-07-22
1    1942-01-08
2    1949-05-14
3    1943-06-07
4    1943-06-27
        ...    
94   1956-05-29
95   1956-05-29
96   1964-07-20
97   2023-06-13
98   1943-03-10
Name: birthdate, Length: 99, dtype: datetime64[ns]
0     [1945-07-22 00:00:00]
1     [1942-01-08 00:00:00]
2     [1949-05-14 00:00:00]
3     [1943-06-07 00:00:00]
4     [1943-06-27 00:00:00]
              ...          
94    [1956-05-29 00:00:00]
95    [1956-05-29 00:00:00]
96    [1964-07-20 00:00:00]
97    [2023-06-13 00:00:00]
98    [1943-03-10 00:00:00]
Name: birthdate, Length: 99, dtype: object

 sex

 education

 BMI
0     33.0
1     33.0
2     27.0
3      0.0
4      0.0
  

In [414]:
dataset.get_all_tokens(key="type")

['Malignant', 'Benign', nan]

In [415]:
dataset.input_info

{'id': {'type': int, 'dim': 2, 'len': 1},
 'hospital': {'type': str, 'dim': 2},
 'birthdate': {'type': int, 'dim': 2, 'len': 1},
 'sex': {'type': str, 'dim': 2},
 'education': {'type': str, 'dim': 2},
 'BMI': {'type': int, 'dim': 2, 'len': 1},
 'smoking': {'type': str, 'dim': 2},
 'packyear_index': {'type': int, 'dim': 2, 'len': 1},
 'air_pollution': {'type': int, 'dim': 2, 'len': 1},
 'fh_cancer': {'type': str, 'dim': 2},
 'ph_cancer': {'type': int, 'dim': 2, 'len': 1},
 'COPD': {'type': int, 'dim': 2, 'len': 1},
 'ASTHMA': {'type': int, 'dim': 2, 'len': 1},
 'bronchiectasis': {'type': int, 'dim': 2, 'len': 1},
 'TB': {'type': int, 'dim': 2, 'len': 1},
 'pneumonia': {'type': int, 'dim': 2, 'len': 1},
 'FVC': {'type': int, 'dim': 2, 'len': 1},
 'FVCL': {'type': int, 'dim': 2, 'len': 1},
 'FEV1': {'type': int, 'dim': 2, 'len': 1},
 'FEV1L': {'type': int, 'dim': 2, 'len': 1},
 'indice': {'type': int, 'dim': 2, 'len': 1},
 'DLCO': {'type': int, 'dim': 2, 'len': 1},
 'id.1': {'type': int, 

In [416]:
import torch 
import torch.nn as nn
from pyhealth.models import BaseModel, TransformerLayer

class Transformer(BaseModel):
    """Transformer model.

    This model applies a separate Transformer layer for each feature, and then
    concatenates the final hidden states of each Transformer layer. The concatenated
    hidden states are then fed into a fully connected layer to make predictions.

    Note:
        We use separate Transformer layers for different feature_keys.
        Currentluy, we automatically support different input formats:
            - code based input (need to use the embedding table later)
            - float/int based value input
        We follow the current convention for the transformer model:
            - case 1. [code1, code2, code3, ...]
                - we will assume the code follows the order; our model will encode
                each code into a vector and apply transformer on the code level
            - case 2. [[code1, code2]] or [[code1, code2], [code3, code4, code5], ...]
                - we will assume the inner bracket follows the order; our model first
                use the embedding table to encode each code into a vector and then use
                average/mean pooling to get one vector for one inner bracket; then use
                transformer one the braket level
            - case 3. [[1.5, 2.0, 0.0]] or [[1.5, 2.0, 0.0], [8, 1.2, 4.5], ...]
                - this case only makes sense when each inner bracket has the same length;
                we assume each dimension has the same meaning; we run transformer directly
                on the inner bracket level, similar to case 1 after embedding table
            - case 4. [[[1.5, 2.0, 0.0]]] or [[[1.5, 2.0, 0.0], [8, 1.2, 4.5]], ...]
                - this case only makes sense when each inner bracket has the same length;
                we assume each dimension has the same meaning; we run transformer directly
                on the inner bracket level, similar to case 2 after embedding table

        dataset: the dataset to train the model. It is used to query certain
            information such as the set of all tokens.
        feature_keys:  list of keys in samples to use as features,
            e.g. ["conditions", "procedures"].
        label_key: key in samples to use as label (e.g., "drugs").
        mode: one of "binary", "multiclass", or "multilabel".
        embedding_dim: the embedding dimension. Default is 128.
        **kwargs: other parameters for the Transformer layer.

    Examples:
        >>> from pyhealth.datasets import SampleEHRDataset
        >>> samples = [
        ...         {
        ...             "patient_id": "patient-0",
        ...             "visit_id": "visit-0",
        ...             "list_codes": ["505800458", "50580045810", "50580045811"],  # NDC
        ...             "list_vectors": [[1.0, 2.55, 3.4], [4.1, 5.5, 6.0]],
        ...             "list_list_codes": [["A05B", "A05C", "A06A"], ["A11D", "A11E"]],  # ATC-4
        ...             "list_list_vectors": [
        ...                 [[1.8, 2.25, 3.41], [4.50, 5.9, 6.0]],
        ...                 [[7.7, 8.5, 9.4]],
        ...             ],
        ...             "label": 1,
        ...         },
        ...         {
        ...             "patient_id": "patient-0",
        ...             "visit_id": "visit-1",
        ...             "list_codes": [
        ...                 "55154191800",
        ...                 "551541928",
        ...                 "55154192800",
        ...                 "705182798",
        ...                 "70518279800",
        ...             ],
        ...             "list_vectors": [[1.4, 3.2, 3.5], [4.1, 5.9, 1.7], [4.5, 5.9, 1.7]],
        ...             "list_list_codes": [["A04A", "B035", "C129"]],
        ...             "list_list_vectors": [
        ...                 [[1.0, 2.8, 3.3], [4.9, 5.0, 6.6], [7.7, 8.4, 1.3], [7.7, 8.4, 1.3]],
        ...             ],
        ...             "label": 0,
        ...         },
        ...     ]
        >>> dataset = SampleEHRDataset(samples=samples, dataset_name="test")
        >>>
        >>> from pyhealth.models import Transformer
        >>> model = Transformer(
        ...         dataset=dataset,
        ...         feature_keys=[
        ...             "list_codes",
        ...             "list_vectors",
        ...             "list_list_codes",
        ...             "list_list_vectors",
        ...         ],
        ...         label_key="label",
        ...         mode="multiclass",
        ...     )
        >>>
        >>> from pyhealth.datasets import get_dataloader
        >>> train_loader = get_dataloader(dataset, batch_size=2, shuffle=True)
        >>> data_batch = next(iter(train_loader))
        >>>
        >>> ret = model(**data_batch)
        >>> print(ret)
        {
            'loss': tensor(4.0555, grad_fn=<NllLossBackward0>),
            'y_prob': tensor([[1.0000e+00, 1.8206e-06],
                        [9.9970e-01, 3.0020e-04]], grad_fn=<SoftmaxBackward0>),
            'y_true': tensor([0, 1]),
            'logit': tensor([[ 7.6283, -5.5881],
                        [ 1.0898, -7.0210]], grad_fn=<AddmmBackward0>)
        }
        >>>

    """

    def __init__(
        self,
        dataset: SampleEHRDataset,
        feature_keys: List[str],
        label_key: str,
        mode: str,
        pretrained_emb: str = None,
        embedding_dim: int = 128,
        **kwargs
    ):
        super(Transformer, self).__init__(
            dataset=dataset,
            feature_keys=feature_keys,
            label_key=label_key,
            mode=mode,
            pretrained_emb=pretrained_emb,
        )
        self.embedding_dim = embedding_dim

        # validate kwargs for Transformer layer
        if "feature_size" in kwargs:
            raise ValueError("feature_size is determined by embedding_dim")

        # the key of self.feat_tokenizers only contains the code based inputs
        self.feat_tokenizers = {}
        self.label_tokenizer = self.get_label_tokenizer()
        # the key of self.embeddings only contains the code based inputs
        self.embeddings = nn.ModuleDict()
        # the key of self.linear_layers only contains the float/int based inputs
        self.linear_layers = nn.ModuleDict()

        # add feature transformation layers
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            # sanity check
            if input_info["type"] not in [str, float, int]:
                raise ValueError(
                    "Transformer only supports str code, float and int as input types"
                )
            elif (input_info["type"] == str) and (input_info["dim"] not in [2, 3]):
                raise ValueError(
                    "Transformer only supports 2-dim or 3-dim str code as input types"
                )
            elif (input_info["type"] in [float, int]) and (
                input_info["dim"] not in [2, 3]
            ):
                raise ValueError(
                    "Transformer only supports 2-dim or 3-dim float and int as input types"
                )
            # for code based input, we need Type
            # for float/int based input, we need Type, input_dim
            self.add_feature_transform_layer(feature_key, input_info)

        self.transformer = nn.ModuleDict()
        for feature_key in feature_keys:
            self.transformer[feature_key] = TransformerLayer(
                feature_size=embedding_dim, **kwargs
            )
        output_size = self.get_output_size(self.label_tokenizer)
        # transformer's output feature size is still embedding_dim
        self.fc = nn.Linear(len(self.feature_keys) * self.embedding_dim, output_size)

    def forward(self, **kwargs) -> Dict[str, torch.Tensor]:
        """Forward propagation.

        The label `kwargs[self.label_key]` is a list of labels for each patient.

        Args:
            **kwargs: keyword arguments for the model. The keys must contain
                all the feature keys and the label key.

        Returns:
            A dictionary with the following keys:
                loss: a scalar tensor representing the loss.
                y_prob: a tensor representing the predicted probabilities.
                y_true: a tensor representing the true labels.
        """
        patient_emb = []
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            dim_, type_ = input_info["dim"], input_info["type"]

            # for case 1: [code1, code2, code3, ...]
            if (dim_ == 2) and (type_ == str):
                x = self.feat_tokenizers[feature_key].batch_encode_2d(
                    kwargs[feature_key]
                )
                # (patient, event)
                x = torch.tensor(x, dtype=torch.long, device=self.device)
                # (patient, event, embedding_dim)
                x = self.embeddings[feature_key](x)
                # (patient, event)
                mask = torch.any(x !=0, dim=2)

            # for case 2: [[code1, code2], [code3, ...], ...]
            elif (dim_ == 3) and (type_ == str):
                x = self.feat_tokenizers[feature_key].batch_encode_3d(
                    kwargs[feature_key]
                )
                # (patient, visit, event)
                x = torch.tensor(x, dtype=torch.long, device=self.device)
                # (patient, visit, event, embedding_dim)
                x = self.embeddings[feature_key](x)
                # (patient, visit, embedding_dim)
                x = torch.sum(x, dim=2)
                # (patient, visit)
                mask = torch.any(x !=0, dim=2)

            # for case 3: [[1.5, 2.0, 0.0], ...]
            elif (dim_ == 2) and (type_ in [float, int]):
                x, mask = self.padding2d(kwargs[feature_key])

                # (patient, event, values)
                x = torch.tensor(x, dtype=torch.float, device=self.device)
                # (patient, event, embedding_dim)
                x = self.linear_layers[feature_key](x)
                # (patient, event)
                mask = mask.bool().to(self.device)

            # for case 4: [[[1.5, 2.0, 0.0], [1.8, 2.4, 6.0]], ...]
            elif (dim_ == 3) and (type_ in [float, int]):
                x, mask = self.padding3d(kwargs[feature_key])
                # (patient, visit, event, values)
                x = torch.tensor(x, dtype=torch.float, device=self.device)
                # (patient, visit, embedding_dim)
                x = torch.sum(x, dim=2)
                x = self.linear_layers[feature_key](x)
                mask = mask[:, :, 0]
                mask = mask.bool().to(self.device)

            else:
                raise NotImplementedError

            # transform x to (patient, event, embedding_dim)
            if self.pretrained_emb != None:
                x = self.linear_layers[feature_key](x)
        
            _, x = self.transformer[feature_key](x, mask, kwargs.get('register_hook'))
            patient_emb.append(x)
           

        patient_emb = torch.cat(patient_emb, dim=1)

        logits = self.fc(patient_emb)
        # obtain y_true, loss, y_prob
        y_true = self.prepare_labels(kwargs[self.label_key], self.label_tokenizer)
        loss = self.get_loss_function()(logits, y_true)
        y_prob = self.prepare_y_prob(logits)
        results = {"loss": loss, "y_prob": y_prob, "y_true": y_true, "logit": logits}
        if kwargs.get("embed", False):
            results["embed"] = patient_emb
        
        return results


In [417]:
model = Transformer(dataset=dataset, 
                    feature_keys=["sex", "BMI",'lobe','global_location','histological_diagnosis'],
                    label_key="type", # imagin que es lo que s'ha de predir
                    mode="multiclass", # no hi ha nateja de dataset i posa que hi ha 3 tipus enlloc de ser binari
                    embedding_dim=128)

In [418]:
len(dataset)

99

In [419]:
from pyhealth.datasets import get_dataloader

train_loader = get_dataloader(dataset, batch_size=16, shuffle=True)


In [420]:
data_batch = next(iter(train_loader))

In [421]:
"""
 # for case 3: [[1.5, 2.0, 0.0], ...]
            elif (dim_ == 2) and (type_ in [float, int]):
                import pdb
                pdb.set_trace()
                x, mask = self.padding2d(kwargs[feature_key])
                # (patient, event, values)
                x = torch.tensor(x, dtype=torch.float, device=self.device)
                # (patient, event, embedding_dim)
                x = self.linear_layers[feature_key](x)
                # (patient, event)
                mask = mask.bool().to(self.device)
"""

# try the model
ret = model(**data_batch)
print(ret)

# try loss backward
ret["loss"].backward()


from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=train_loader,
    epochs=10
)

{'loss': tensor(6.3571, grad_fn=<NllLossBackward0>), 'y_prob': tensor([[8.1600e-01, 1.9084e-04, 1.8381e-01],
        [1.0112e-04, 3.8071e-01, 6.1919e-01],
        [3.1708e-03, 2.3202e-02, 9.7363e-01],
        [1.2581e-01, 6.4364e-01, 2.3056e-01],
        [8.7273e-03, 2.5589e-01, 7.3538e-01],
        [2.3582e-04, 4.4769e-04, 9.9932e-01],
        [1.0873e-02, 1.6904e-04, 9.8896e-01],
        [2.7010e-02, 5.7165e-02, 9.1583e-01],
        [3.0922e-05, 2.9703e-02, 9.7027e-01],
        [1.5858e-01, 2.6992e-02, 8.1443e-01],
        [1.8179e-05, 3.1021e-06, 9.9998e-01],
        [1.2252e-02, 1.6962e-01, 8.1813e-01],
        [1.5402e-02, 9.8256e-07, 9.8460e-01],
        [6.7435e-04, 4.2046e-03, 9.9512e-01],
        [3.3718e-04, 1.4483e-08, 9.9966e-01],
        [3.1748e-03, 6.9984e-05, 9.9676e-01]], grad_fn=<SoftmaxBackward0>), 'y_true': tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]), 'logit': tensor([[10.2575,  1.8967,  8.7670],
        [-4.5506,  3.6829,  4.1693],
        [-2.5864, -0

Epoch 0 / 10: 100%|██████████| 7/7 [00:00<00:00, 23.09it/s]

--- Train epoch-0, step-7 ---
loss: 2.6255



Evaluation: 100%|██████████| 7/7 [00:00<00:00, 42.80it/s]

--- Eval epoch-0, step-7 ---


...

--- Eval epoch-8, step-63 ---
accuracy: 0.9899
f1_macro: 0.6645
f1_micro: 0.9899
loss: 0.0439




Epoch 9 / 10: 100%|██████████| 7/7 [00:00<00:00, 30.65it/s]

--- Train epoch-9, step-70 ---
loss: 0.0813



Evaluation: 100%|██████████| 7/7 [00:00<00:00, 68.14it/s]

--- Eval epoch-9, step-70 ---
accuracy: 1.0000
f1_macro: 1.0000
f1_micro: 1.0000
loss: 0.0057


#### COSES QUE HE FET PERO QUE NO SERVEIXEN EN GENERAL PER RES

In [16]:
from pyhealth.tokenizer import Tokenizer
print(radiolung_df["sex"].unique())
token_space = radiolung_df[["fh_cancer"]].to_numpy().flatten().tolist()
print(token_space
      )

['Woman' ' Man']
[' No', ' No', ' No', ' No', ' No', ' No', ' No', ' (Yes) others', ' (Yes) others', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' (Yes) others', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' (Yes) others', ' No', ' No', ' No', ' No', '(Yes) Lung cancer', ' (Yes) others', ' No', '(Yes) Lung cancer', ' No', ' (Yes) others', '(Yes) Lung cancer', ' No', ' (Yes) others', ' No', ' (Yes) others', ' No', ' No', '(Yes) Lung cancer', ' (Yes) others', ' (Yes) others', ' No', ' No', ' No', ' (Yes) others', '(Yes) others', '(Yes) Lung cancer', nan, ' No', ' No', nan, '(Yes) others', '(Yes) Lung cancer', ' No', ' No', nan, ' No', ' No', '(Yes) others', '(Yes) Lung cancer', '(Yes) Lung cancer', ' No', ' No', '(Yes) others', ' No', ' No', '(Yes) others', nan, ' No', '(Yes) Lung cancer', '(Yes) others', ' No', ' No', nan, '(Yes) others', ' No', nan, nan, '(Yes) others', ' No', nan, '(Yes) others', nan, '(Yes) Lung cancer', ' No', ' No', ' No', '(Yes) others', ' No', '(

In [17]:
tokenizer = Tokenizer(tokens=token_space, special_tokens=["<pad>", "<unk>"])

In [18]:
tokenizer.vocabulary.token2idx

{'<pad>': 0,
 '<unk>': 1,
 ' No': 2,
 ' (Yes) others': 3,
 '(Yes) Lung cancer': 4,
 '(Yes) others': 5,
 nan: 6}

In [19]:
tokenizer.convert_tokens_to_indices(["hola", "pepe"])

[1, 1]